In [65]:
# First, we can import some functions so we can use the model
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite

# Second, we can read the GEM and save it as ‘model’
model = read_sbml_model('data/iML1515_het.xml')

In [76]:
#phaC
with model:
    r_phaC_exp = model.add_boundary(model.metabolites.get_by_id('P3HB_c'), type='demand')
    model.objective = r_phaC_exp
    solution = model.optimize()

phaC_flux = solution.fluxes["DM_P3HB_c"]
print(phaC_flux)
growth = solution.objective_value
with model:
    model.reactions.phaC.lower_bound = phaC_flux * 2
    solution_overexpression = model.optimize()
    growth_overexpression = solution_overexpression.objective_value
    
growth_reduction = growth - growth_overexpression
print(growth)
print(phaC_flux)
print(f"Growth reduction is: {growth_reduction}")


12.28313043478247
12.28313043478247
12.28313043478247
Growth reduction is: 12.28313043478247


C:\Users\bierk\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [ ]:
solution = model.optimize()
solution.objective_value #Biomass is the objective value by default

In [95]:
with model:
    max_growth = model.optimize().objective_value
    min_growth = 0.8 * max_growth  
    max_growth = 1.0 * max_growth  
    print(min_growth, max_growth)
    model.objective = model.reactions.get_by_id("BIOMASS_Ec_iML1515_core_75p37M")
    model.reactions.get_by_id("BIOMASS_Ec_iML1515_core_75p37M").bounds = min_growth, max_growth
    max_growth = model.optimize().objective_value
    print(max_growth)
    
    r_phaC_exp = model.add_boundary(model.metabolites.get_by_id('P3HB_c'), type='sink')
    model.objective = r_phaC_exp
    print(model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound)
    produce_P3HB = model.optimize().objective_value

print(produce_P3HB)


0.7015977715415835 0.8769972144269793
0.8769972144269795
0.7015977715415835
2.7428235294117687
